# 0. Import libraries

In [1]:
import time
import copy
import numpy as np
from tqdm import trange
from matplotlib import pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import warnings
warnings.filterwarnings(action="ignore")
#from google.colab import drive
#drive.mount('/content/drive')

In [2]:
CUDA = torch.cuda.is_available()
CUDA_DEVICE = 0
if CUDA: device='cuda'
else: device='cpu'
device

'cpu'

In [4]:
def activation(Tensor):
    return torch.tanh(Tensor).to(device)

In [5]:
class RitzBlock(nn.Module):
    def __init__(self, n_hid):
        super(RitzBlock, self).__init__()
        self.n_hid = n_hid
        ##Layer 1
        self.Weight_layer1 = nn.Parameter(torch.rand(size=(n_hid,n_hid),requires_grad=True)).to(device)
        self.Bias_layer1 = nn.Parameter(torch.rand(size=(1,n_hid),requires_grad=True)).to(device)
        ##Layer 2
        self.Weight_layer2 = nn.Parameter(torch.rand(size=(n_hid,n_hid),requires_grad=True)).to(device)
        self.Bias_layer2 = nn.Parameter(torch.rand(size=(1,n_hid),requires_grad=True)).to(device)

    #Activation function
    def layer1(self,Tensor): return activation(torch.matmul(Tensor,self.Weight_layer1)+self.Bias_layer1)
    def layer2(self,Tensor): return activation(torch.matmul(Tensor,self.Weight_layer2)+self.Bias_layer2)
    
    #Forward process
    def forward(self,Tensor):
        h1=self.layer1(Tensor).to(device)
        h2=self.layer2(h1).to(device)
        return h2+Tensor #return.shape = (batch_size,tlength+1,nhid)

In [6]:
class PreModel(nn.Module):
    def __init__(self,n_hid):
        super(PreModel,self).__init__()
        self.n_hid=n_hid
        self.pre_Weight=nn.Parameter(torch.rand(size=(1+nptl,n_hid),requires_grad=True)).to(device)
        self.pre_Bias=nn.Parameter(torch.rand(size=(1,n_hid),requires_grad=True)).to(device)
        #dof : 자유도     
    def forward(self,Tensor):
        if self.n_hid>nptl+1: 
            pad=torch.nn.ZeroPad2d((0,n_hid-1-nptl,0,0))
            output=pad(Tensor)
        else:
            output=activation(torch.matmul(Tensor,self.pre_Weight)+self.pre_Bias)
        return output.to(device)

In [ ]:
class NeuralNetworks(nn.Module):
    def __init__(self,n_block,n_hid):
        super(NeuralNetworks, self).__init__()
        self.n_blocks=n_block
        self.n_hid=n_hid
        
        self.nn_Weight=nn.Parameter(torch.rand(size=(n_hid,nptl),requires_grad=True)).to(device)
        self.nn_Bias=nn.Parameter(torch.rand(size=(1,nptl),requires_grad=True)).to(device)        
        
        self.premodel=PreModel(self.n_hid)
        
        self.Ritzblocks=nn.ModuleList()
        for _ in range(self.n_blocks): self.Ritzblocks.append(RitzBlock(self.n_hid))
    
    def Observable(self,Tensor):
        position0=Tensor[...,:-1,...]
        batch_size=Tensor.shape[0]
        
        
        return torch.mean(Obs)

    def Girsanov_weight(self,force):        
        #force.shape=(ntraj,tlength+1,4*nptl)
        F_r=force[...,:-1,:2*nptl]
        F_v=force[...,:-1,2*nptl:]
        iteration=force.shape[0]
        position_part=dt*torch.bmm(F_r,torch.permute(F_r,(0,2,1)))
        velocity_part=dt*torch.bmm(F_v,torch.permute(F_v,(0,2,1)))
        Girsanov=torch.zeros(iteration)
        for n in range(iteration):
            Girsanov[n]=torch.trace(position_part[n,...]/DT+velocity_part[n,...]/DR)
        return -0.5*torch.mean(Girsanov)

    #def loss(self,Tensor,noise,force):
    #    L=Lambda*(dt*tlength)*self.Observable(Tensor,noise)+self.Girsanov_weight(force)
    #    return L

    def forward(self,Tensor):
        T=self.premodel(Tensor).to(device)
        for model in self.Ritzblocks:
            val=model(T); T=val
            print()
        del_u=torch.matmul(T,self.nn_Weight)+self.nn_Bias
        return del_u #variational force